<a href="https://colab.research.google.com/github/dbremont/Notas/blob/main/Papers/computation/XFS%3A_the_big_storage_file_system_for_Linux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[XFS: the big storage file system for Linux](https://www.usenix.org/system/files/login/articles/140-hellwig.pdf)

[xfs code](https://git.kernel.org/pub/scm/fs/xfs/xfs-linux.git/about/)

[files systems impl linux code](https://github.com/torvalds/linux/tree/master/fs)

XFS is a "file system" that was designed from day one for computer systems with large numbers of CPUs and large disk arrays. It focuses on supporting large files and good streaming I/O performace.

XFS features:
- Delayed allocations,
- Extends

**Space Allocation and Management**

Each XFS file syste is partitioned into regions called allocations groups (AGS). Allocations groups are somewhat similar to block groups in ext3, but Ags typically much larger block groups and are used for scalability and parallelism rather than disk locality.

Allocation groups are typically sized between 0.5 and 4 gigabytes and keep the size of the XFS data structures in a range where they can operate efficiently  and in parallel.

Historical Unix file systems, includign ext3, use lienar bitma;s to track free space, which is inefficient specialy for larger contingous allocations. XFS instead uses a parir of B+ trees in each allocation group. Each entry in the B+ tree nodes consist of a start block and length pair describin ga free-space region. The first B+ tree is indexed by the starting block of the free region, and the other is indexed by the length of the free region. The double indexing allows the allocator to consider two goals for new data placement: locality to existing file data, and best fit into free space.

A similar **extend concept** is used for tracking the disk blocks assigned to each fie. In addition to the start block of disk and the length of the contitnous range, the extend descritor also contain two additional fields.  The first  one is the logical offset in to the file, which alows for efficien tsparse file suport by skipping ranges that do not have blocks allocated to them. The second one is a simple one-bit flag to mark extend as unwritten.


**Inodes and Extended Attributed**

The XFS inode consists of three parts: the inode core, the data fork, and the optional attribute fork. The inode conre contains traditional `UNIX inode` metada such as owner group, number of blocks, timestamps, and a few XFS-specific additonal such as project ID.

**Directories**

XFS supports two major forms of directories. If a directory contains only a few entries and is small enough to fit into the inode, a simple unsorted linear format can store all data inide the inode's data fork.  The advantge of this format is that no external block is used and access to the directoy is extremely fast, sinse it will already be complelty cached in memory once itt is accesed. Linear algorithms, howerver, do not scale to large directoires with millions of entires. XFS thus again uses B+ trees to manage large directoires.

**I/O Scalability**

`Delayed allocation` technique is explore to increase large contigous regions. 

**Direct I/O**

XFS provides a featured, called direct I/O, that provides the semantics of a UNIX raw device inside the file system namesspace. 

**Crash Recovery**

For today's large file systems, a full file system check on an unclean shudown is not acceptable becuase it would take too long. To avoid the requirement for regular file system checks, XFS use a write-ahead loggin scheme that enables atomic updates of the file system. XFS only logs structural updates to the file system metadata, but not the actual user data, for which the `Posix file system interface` does ont provide useful atomicity guarantees.

XFS logs every update to the file system data structures and does nto batch changes from multiple transactions into a single log write, as is done by ext3. This means that XFS must write significantlly more data to the log in case a single metadata structure gets modified again and again in short sequence(e.g removing a large number of small files). To mitigate the impact of log writes to the system performace, an external lgo device can be used.  With an external log the additional seeks on the main device are reduced, and the log can use the full sequential performace of the log device.

Unfortunately, transaction loggin does not help to protect against hardware-induced erros. To deal with these problems, XFS has an offline file sysstem checking and reapr tool caled `xfs_repair`. To deal with the ever growing disk sized and worsenign seek rates, `xfs_repair` has underdon ea major overhaul in the past few year to perform efficient read-ahead and chachign and to make use of multiple processors in SMP systems.

**Disk Quotes**

XFS provides an enhance implementation of the BSD disk quotas. It supports the normal soft and hard limits for disk space usage and number of inodes as an intergral part of the file system. 

`xfs_quota`

**Day-to-Day Use**

A  file system is used should be boring and mostly invisible of the system administrator and user. But to get to that state the file system must first be crated.  An xfs file system is creted with the mkfs.xfs command, which is trivial to use:

`mkfs.xfs /dev/vg00/scratch`





## References

[1] Adam Sweeney et al., “Scalability in the XFS File System,” Proceedings
of the USENIX 1996 Annual Technical Conference.

[2] Silicon Graphics Inc., XFS Filesystem Structure, 2nd edition,
http://oss.sgi.com/projects/xfs/papers/xfs_filesystem_structure.pdf.

[3] Linux attr(5) man page: http://linux.die.net/man/5/attr.

[4] Dave Chinner and Jeremy Higdon, “Exploring High Bandwidth Filesystems on Large Systems,” Proceedings of the Ottawa Linux Symposium 2006:
http://oss.sgi.com/projects/xfs/papers/ols2006/ols-2006-paper.pdf.

[5] Silicon Graphics Inc., XFS Overview and Internals: http://oss.sgi.com/
projects/xfs/training/index.html.

[6] Dave Chinner and Barry Naujok, Fixing XFS Filesystems Faster: http://
mirror.linux.org.au/pub/linux.conf.au/2008/slides/135-fixing_xfs_faster.pdf.

[7] Dr. Stephen Tweedie, “EXT3, Journaling Filesystem,” transcript of a presentation at the Ottawa Linux Symposium 2000: http://olstrans.sourceforge.
net/release/OLS2000-ext3/OLS2000-ext3.html.
[8] xfs_quota(8)—Linux manpage: http://linux.die.net/man/8/xfs_quota.

## Questions?

What is a file system?

How many file systems does linux support?

Explore the pluging architecture 

What is the meaning of a "mount a file system"?

What are the file systems that linux support?

Can the number of files systems  in my computer be larger than the number of files systems that linux support?